In [1]:
from pynq import Overlay
from pynq import allocate
import time
import numpy as np

In [2]:
# Load Overlay
overlay = Overlay('./Overlay/cnna.bit')

In [3]:
overlay?

In [4]:
test_ip = overlay.cnna_0

In [5]:
test_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  in_data_1 = Register(in_data=0),
  in_data_2 = Register(in_data=0),
  in_weights_1 = Register(in_weights=0),
  in_weights_2 = Register(in_weights=0),
  out_r_i = Register(out_r_i=0),
  out_r_o = Register(out_r_o=0),
  out_r_o_ctrl = Register(out_r_o_ap_vld=0, RESERVED=0)
}

```c
// ==============================================================
// Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2020.1 (64-bit)
// Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.
// ==============================================================
// control
// 0x00 : Control signals
//        bit 0  - ap_start (Read/Write/COH)
//        bit 1  - ap_done (Read/COR)
//        bit 2  - ap_idle (Read)
//        bit 3  - ap_ready (Read)
//        bit 7  - auto_restart (Read/Write)
//        others - reserved
// 0x04 : Global Interrupt Enable Register
//        bit 0  - Global Interrupt Enable (Read/Write)
//        others - reserved
// 0x08 : IP Interrupt Enable Register (Read/Write)
//        bit 0  - enable ap_done interrupt (Read/Write)
//        bit 1  - enable ap_ready interrupt (Read/Write)
//        others - reserved
// 0x0c : IP Interrupt Status Register (Read/TOW)
//        bit 0  - ap_done (COR/TOW)
//        bit 1  - ap_ready (COR/TOW)
//        others - reserved
// 0x10 : Data signal of in_data
//        bit 31~0 - in_data[31:0] (Read/Write)
// 0x14 : Data signal of in_data
//        bit 31~0 - in_data[63:32] (Read/Write)
// 0x18 : reserved
// 0x1c : Data signal of in_weights
//        bit 31~0 - in_weights[31:0] (Read/Write)
// 0x20 : Data signal of in_weights
//        bit 31~0 - in_weights[63:32] (Read/Write)
// 0x24 : reserved
// 0x28 : Data signal of out_r_i
//        bit 31~0 - out_r_i[31:0] (Read/Write)
// 0x2c : reserved
// 0x30 : Data signal of out_r_o
//        bit 31~0 - out_r_o[31:0] (Read)
// 0x34 : Control signal of out_r_o
//        bit 0  - out_r_o_ap_vld (Read/COR)
//        others - reserved
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)
```

In [6]:
in1 = allocate(shape=(3,3), dtype='int8')
in2 = allocate(shape=(3,3,3), dtype='int8')
print(in1.shape)
#print(in1.physical_address)
print(in2.shape)
#print(in2.physical_address)

(3, 3)
(3, 3, 3)


In [7]:
tarr1 = np.array([[5,5,5],[5,5,5],[5,5,5]], np.int8)
tarr2 = np.array([
    [[2,2,2],[3,3,3],[4,4,4]],
    [[3,3,3],[4,4,4],[5,5,5]],
    [[4,4,4],[5,5,5],[6,6,6]]], np.int8)

print(tarr1)
print("= = = = =")
print(tarr2)

[[5 5 5]
 [5 5 5]
 [5 5 5]]
= = = = =
[[[2 2 2]
  [3 3 3]
  [4 4 4]]

 [[3 3 3]
  [4 4 4]
  [5 5 5]]

 [[4 4 4]
  [5 5 5]
  [6 6 6]]]


In [8]:
#np.copyto(in1, tarr1)
#np.copyto(in2, tarr2)

In [9]:
for i in range(3):
    for j in range(3):
        in1[i][j] = tarr1[i][j]
    
for i in range(3):
    for j in range(3):
        for k in range(3):
            in2[i][j][k] = tarr2[i][j][k]
            
print(in1)
print('\n= = = = = =\n')
print(in2)

[[5 5 5]
 [5 5 5]
 [5 5 5]]

= = = = = =

[[[2 2 2]
  [3 3 3]
  [4 4 4]]

 [[3 3 3]
  [4 4 4]
  [5 5 5]]

 [[4 4 4]
  [5 5 5]
  [6 6 6]]]


In [10]:
# Allocate memory to store output
out = allocate(shape=(1,), dtype='int8')
print(out)

[0]


In [11]:
in1s = 0x10
in2s = 0x1c
outs = 0x30

t1 = time.time()

i = 0

'''
while i < 9:
    if(i%4 == 0):
        print(">at offset",hex(i))
        if(i+4 < in1.shape[0]):
            tdat = in1[i:i+4].tobytes()
        else:
            tdat = in1[i:].tobytes()
            
        print(tdat)
        test_ip.write(in1s+i, tdat)
        
        if(i+4 < in1.shape[0]):
            tdat = in2[i:i+4].tobytes()
        else:
            tdat = in2[i:].tobytes()
            
        print(tdat)
        test_ip.write(in2s+i, tdat)
    i += 1
'''


#test_ip.register_map.Memory_in_data = in1
#test_ip.register_map.Memory_in_weights = in2

test_ip.write(in1s, in1.physical_address)
test_ip.write(in2s, in2.physical_address)
test_ip.write(outs, out.physical_address)
test_ip.write(0x00, 1)

t2 = time.time()

print(t2-t1)
print("out:", out)

0.0011816024780273438
out: [0]


In [13]:
print("out:", out)

out: [0]
